<a href="https://colab.research.google.com/github/mraj2018/AIML/blob/master/R8_External_Lab_Questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


In [0]:
#Importing important modules
import keras
import numpy as np
import keras.utils as np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,Activation
from keras.datasets import cifar10

### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(y_train.shape[0],y_test.shape[0])

50000 10000


In [0]:
x_train04 = []
x_test04 = []
x_train59 = []
x_test59 = []
y_train04 = []
y_test04 = []
y_train59 = []
y_test59 = []

for ix in range(y_train.shape[0]):
    if y_train[ix] < 5:
        # put data in set 1
        x_train04.append(x_train[ix]/255.0)
        y_train04.append(y_train[ix])
    else:
        # put data in set 2
        x_train59.append(x_train[ix]/255.0)
        y_train59.append(y_train[ix])

for ix in range(y_test.shape[0]):
    if y_test[ix] < 5:
        # put data in set 1
        x_test04.append(x_test[ix]/255.0)
        y_test04.append(y_test[ix])
    else:
        # put data in set 2
        x_test59.append(x_test[ix]/255.0)
        y_test59.append(y_test[ix])


x_train04 = np.asarray(x_train04).reshape((-1, 32, 32, 3))
x_test04 = np.asarray(x_test04).reshape((-1, 32, 32, 3))
x_train59 = np.asarray(x_train59).reshape((-1, 32, 32, 3))
x_test59 = np.asarray(x_test59).reshape((-1, 32, 32, 3))

print(x_train04.shape,x_test04.shape)
print(x_train59.shape ,x_test59.shape)


(25000, 32, 32, 3) (5000, 32, 32, 3)
(25000, 32, 32, 3) (5000, 32, 32, 3)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train04 = np_utils.to_categorical(np.asarray(y_train04),5)
y_test04 = np_utils.to_categorical(np.asarray(y_test04), 5)

y_train59= np.asarray(y_train59)-5
y_test59= np.asarray(y_test59)-5
y_train59 = np_utils.to_categorical(y_train59,5)
y_test59 = np_utils.to_categorical(y_test59, 5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
model = Sequential()

model.add(Conv2D(32, 3, 3, input_shape=(32, 32, 3), activation='relu'))
model.add(Conv2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
#model.add(Dropout(0.25))


model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train04, y_train04,
         epochs=10,
         batch_size=32,
         validation_data=(x_test04, y_test04))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_13 (Activation)   (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 12, 12, 64)        36928     
__________

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train59, y_train59,
          batch_size=32,
          epochs=10,
          verbose=1,
          validation_data=(x_test59, y_test59))

Train on 25000 samples, validate on 5000 samples
Epoch 1/10
25000/25000 [==============================] - 7s 284us/step - loss: 0.8094 - acc: 0.7341 - val_loss: 0.4585 - val_acc: 0.8352
Epoch 2/10
25000/25000 [==============================] - 7s 265us/step - loss: 0.4383 - acc: 0.8424 - val_loss: 0.4133 - val_acc: 0.8534
Epoch 3/10
25000/25000 [==============================] - 7s 268us/step - loss: 0.3767 - acc: 0.8627 - val_loss: 0.3900 - val_acc: 0.8564
Epoch 4/10
25000/25000 [==============================] - 7s 264us/step - loss: 0.3288 - acc: 0.8806 - val_loss: 0.3821 - val_acc: 0.8608
Epoch 5/10
25000/25000 [==============================] - 7s 265us/step - loss: 0.2902 - acc: 0.8918 - val_loss: 0.3861 - val_acc: 0.8642
Epoch 6/10
25000/25000 [==============================] - 7s 264us/step - loss: 0.2566 - acc: 0.9077 - val_loss: 0.3796 - val_acc: 0.8672
Epoch 7/10
25000/25000 [==============================] - 7s 264us/step - loss: 0.2223 - acc: 0.9189 - val_loss: 0.3833 - v

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [0]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [0]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [0]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
tfidf_matrix = tf.fit_transform(twenty_train.data)
tfidf_matrix.shape

(2257, 35788)

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
clf = LogisticRegression()
clf.fit(tfidf_matrix, twenty_train.target)

tfidf_matrix_test = tf.transform(twenty_test.data)

y_pred_class = clf.predict(tfidf_matrix_test)
# calculate accuracy
print(metrics.accuracy_score(twenty_test.target, y_pred_class) * 100, '%')

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


88.68175765645806 %
